# Data Generation Workbench

## Purpose

This notebook provides an interactive workbench for generating synthetic AM-QADF datasets. Use widgets to configure data generation parameters, preview generated data, and populate MongoDB collections - all without writing code!

## Learning Objectives

By the end of this notebook, you will:
- ✅ Configure STL processing and hatching generation parameters
- ✅ Generate laser parameters, ISPM data, and CT scan data interactively
- ✅ Preview generated data before committing to MongoDB
- ✅ Populate MongoDB collections with generated datasets
- ✅ Understand the relationship between parameters and generated data
- ✅ Create reproducible datasets using random seeds

## Estimated Duration

60-90 minutes

---

## Overview

The Data Generation Workbench enables interactive creation of synthetic AM-QADF datasets:

- 📐 **STL Processing**: Load and process 3D STL models
- 🛤️ **Hatching Generation**: Generate layer-by-layer scan paths
- ⚡ **Laser Parameters**: Create realistic laser process parameters
- 🌡️ **ISPM Data**: Generate in-situ process monitoring data
- 🔬 **CT Scans**: Create computed tomography scan data
- 💾 **MongoDB Population**: Store generated data in the warehouse

---

## Workflow

**Important**: Follow these steps in order:

1. **Load STL File**: Click "Load STL" button to load and process the selected STL model
2. **Configure Parameters**: Adjust parameters in each accordion section as needed
3. **Select Collections**: Choose which MongoDB collections to populate
4. **Preview** (optional): Click "Preview" to see what will be generated without saving
5. **Generate**: Once ready, the "Generate" button will be enabled - click it to create and save data

The **Generate** button will remain disabled until:
- ✅ An STL file is loaded
- ✅ At least one collection is selected for population

---

Use the interactive widgets below to explore data generation - all parameters are adjustable in real-time!


In [7]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText, SelectMultiple,
    HTML as WidgetHTML, interactive, interact, fixed, Textarea
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from typing import Optional, Tuple, Dict, Any, List, Callable
import json
import time

# Try to import infrastructure
INFRASTRUCTURE_AVAILABLE = False
try:
    from src.infrastructure.database import get_connection_manager
    from src.infrastructure.config import MongoDBConfig
    INFRASTRUCTURE_AVAILABLE = True
except Exception as e:
    print(f"⚠️ Infrastructure layer not available: {type(e).__name__}")

# Try to import generation modules
GENERATION_AVAILABLE = False
try:
    from generation.process.stl_processor import STLProcessor
    from generation.process.hatching_generator import HatchingGenerator, HatchingConfig
    from generation.sensors.laser_parameter_generator import LaserParameterGenerator, LaserParameterGeneratorConfig
    from generation.sensors.ispm_generator import ISPMGenerator, ISPMGeneratorConfig
    from generation.sensors.ct_scan_generator import CTScanGenerator, CTScanGeneratorConfig
    GENERATION_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ Generation modules not available: {e}")
    GENERATION_AVAILABLE = False

# Initialize MongoDB connection
mongo_client = None
if INFRASTRUCTURE_AVAILABLE:
    try:
        manager = get_connection_manager(env_name="development")
        mongo_client = manager.get_mongodb_client()
        if mongo_client and mongo_client.is_connected():
            print("✅ MongoDB connection established")
        else:
            print("⚠️ MongoDB connection failed")
    except Exception as e:
        print(f"⚠️ MongoDB connection error: {type(e).__name__}: {e}")

print("✅ Setup complete!")
print("\n" + "="*60)
print("NOTEBOOK STRUCTURE:")
print("="*60)
print("Cell 0: Header and Overview")
print("Cell 1: Setup and Imports (you just ran this)")
print("Cell 2: Widget Definitions Guide")
print("Cells 3-19: ALL Widget Definitions (run these)")
print("Cell 20: Display Interface (run this)")
print("Cells 21-27: ALL Event Handlers (run these)")
print("="*60)


✅ MongoDB connection established
✅ Setup complete!

NOTEBOOK STRUCTURE:
Cell 0: Header and Overview
Cell 1: Setup and Imports (you just ran this)
Cell 2: Widget Definitions Guide
Cells 3-19: ALL Widget Definitions (run these)
Cell 20: Display Interface (run this)
Cells 21-27: ALL Event Handlers (run these)


## Quick Start Guide

**To use this notebook, run cells in this order:**

1. ✅ **Cell 0**: Read the header (already done)
2. ✅ **Cell 1**: Setup and imports (already done)  
3. 📦 **Cells 3-19**: Widget definitions - Run all these cells to create the interface
4. 🖥️ **Cell 20**: Display interface - Run this to show the interface
5. ⚙️ **Cells 21-27**: Event handlers - Run all these to connect functionality

**Or simply: Run all cells from top to bottom!**

---

The widget definitions are organized into logical sections, but they all work together as a single unified interface.


In [8]:
# Part 1: Top Panel - Generation Mode and Model Selection

# Generation mode selector
generation_mode = RadioButtons(
    options=[
        ('Single Model', 'single'),
        ('Batch', 'batch'),
        ('Custom Scenario', 'custom'),
        ('Quick Demo', 'demo')
    ],
    value='single',
    description='Mode:',
    layout=Layout(width='200px')
)

# STL model selector
stl_models_dir = project_root / 'generation' / 'models'
available_stl_files = []
if stl_models_dir.exists():
    available_stl_files = sorted([f.name for f in stl_models_dir.glob('*.stl')])

stl_selector = Dropdown(
    options=available_stl_files if available_stl_files else ['No STL files found'],
    value=available_stl_files[0] if available_stl_files else None,
    description='STL Model:',
    layout=Layout(width='300px'),
    disabled=len(available_stl_files) == 0
)

# Action buttons
# Generate button - disabled until STL is loaded and parameters configured
generate_button = Button(
    description='Generate',
    button_style='success',
    icon='play',
    layout=Layout(width='120px'),
    disabled=True,  # Disabled until ready
    tooltip='Load STL and configure parameters first'
)

# Preview button - can be used to preview configuration
preview_button = Button(
    description='Preview',
    button_style='info',
    icon='eye',
    layout=Layout(width='120px'),
    disabled=False,  # Can preview configuration anytime
    tooltip='Preview generation configuration'
)

clear_preview_button = Button(
    description='Clear Preview',
    button_style='warning',
    icon='trash',
    layout=Layout(width='140px')
)

save_config_button = Button(
    description='Save Config',
    button_style='',
    icon='save',
    layout=Layout(width='140px')
)

# Add info label about Generate button state
generate_status_label = WidgetHTML(
    value='<i style="color:gray; font-size:0.9em;">⚠️ Load STL file to enable Generate</i>',
    layout=Layout(margin='0 10px')
)

# Top panel layout
top_panel = VBox([
    HBox([
        generation_mode,
        stl_selector,
        generate_button,
        preview_button,
        clear_preview_button,
        save_config_button
    ], layout=Layout(
        justify_content='flex-start',
        align_items='center',
        padding='5px'
    )),
    HBox([generate_status_label], layout=Layout(padding='5px'))
], layout=Layout(
    padding='10px',
    border='1px solid #ddd',
    margin='5px'
))

# Top panel created (will be displayed later)


In [9]:
# Part 2: Left Panel - Configuration Accordion
# Section 1: STL Processing Configuration

stl_load_button = Button(
    description='Load STL',
    button_style='info',
    icon='upload',
    layout=Layout(width='120px')
)

stl_preview_toggle = Checkbox(
    value=True,
    description='Show 3D Preview',
    layout=Layout(width='150px')
)

stl_extract_metadata = Checkbox(
    value=True,
    description='Extract Metadata',
    layout=Layout(width='150px')
)

stl_calc_bbox = Checkbox(
    value=True,
    description='Calculate BBox',
    layout=Layout(width='150px')
)

stl_calc_volume = Checkbox(
    value=True,
    description='Calculate Volume',
    layout=Layout(width='150px')
)

stl_metadata_display = Textarea(
    value='No STL loaded',
    description='Metadata:',
    layout=Layout(width='100%', height='150px'),
    disabled=True
)

stl_config_panel = VBox([
    WidgetHTML("<b>STL Processing Configuration</b>"),
    HBox([stl_load_button, stl_preview_toggle]),
    HBox([stl_extract_metadata, stl_calc_bbox, stl_calc_volume]),
    stl_metadata_display
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))


In [10]:
# Part 3: Hatching Generation Configuration

layer_thickness = FloatSlider(
    value=0.05,
    min=0.01,
    max=0.5,
    step=0.01,
    description='Layer Thickness (mm):',
    layout=Layout(width='100%')
)

hatch_spacing = FloatSlider(
    value=0.1,
    min=0.01,
    max=1.0,
    step=0.01,
    description='Hatch Spacing (mm):',
    layout=Layout(width='100%')
)

hatch_angle = FloatSlider(
    value=10.0,
    min=0.0,
    max=180.0,
    step=1.0,
    description='Hatch Angle (deg):',
    layout=Layout(width='100%')
)

contour_offset = FloatSlider(
    value=0.0,
    min=0.0,
    max=0.5,
    step=0.01,
    description='Contour Offset (mm):',
    layout=Layout(width='100%')
)

max_layers = IntSlider(
    value=50,
    min=1,
    max=1000,
    step=1,
    description='Max Layers:',
    layout=Layout(width='100%')
)

sampling_spacing = FloatSlider(
    value=1.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description='Sampling Spacing (mm):',
    layout=Layout(width='100%')
)

laser_beam_width = FloatSlider(
    value=0.1,
    min=0.01,
    max=0.5,
    step=0.01,
    description='Laser Beam Width (mm):',
    layout=Layout(width='100%')
)

hatch_pattern = RadioButtons(
    options=['Raster', 'Stripe', 'Chessboard'],
    value='Raster',
    description='Pattern:',
    layout=Layout(width='100%')
)

preview_hatching_button = Button(
    description='Preview Hatching',
    button_style='info',
    icon='eye',
    layout=Layout(width='150px')
)

hatching_stats_display = WidgetHTML(value='<i>No hatching generated yet</i>')

hatching_config_panel = VBox([
    WidgetHTML("<b>Hatching Generation Configuration</b>"),
    layer_thickness,
    hatch_spacing,
    hatch_angle,
    contour_offset,
    max_layers,
    sampling_spacing,
    laser_beam_width,
    hatch_pattern,
    preview_hatching_button,
    hatching_stats_display
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))


In [11]:
# Part 4: Laser Parameters Configuration

# Power configuration
power_mean = FloatSlider(
    value=200.0,
    min=50.0,
    max=500.0,
    step=5.0,
    description='Mean Power (W):',
    layout=Layout(width='100%')
)

power_std = FloatSlider(
    value=10.0,
    min=0.0,
    max=50.0,
    step=1.0,
    description='Std Power (W):',
    layout=Layout(width='100%')
)

power_min = FloatSlider(
    value=100.0,
    min=50.0,
    max=400.0,
    step=5.0,
    description='Min Power (W):',
    layout=Layout(width='100%')
)

power_max = FloatSlider(
    value=400.0,
    min=100.0,
    max=500.0,
    step=5.0,
    description='Max Power (W):',
    layout=Layout(width='100%')
)

# Speed configuration
speed_mean = FloatSlider(
    value=500.0,
    min=100.0,
    max=3000.0,
    step=50.0,
    description='Mean Speed (mm/s):',
    layout=Layout(width='100%')
)

speed_std = FloatSlider(
    value=50.0,
    min=0.0,
    max=200.0,
    step=5.0,
    description='Std Speed (mm/s):',
    layout=Layout(width='100%')
)

speed_min = FloatSlider(
    value=100.0,
    min=50.0,
    max=2000.0,
    step=50.0,
    description='Min Speed (mm/s):',
    layout=Layout(width='100%')
)

speed_max = FloatSlider(
    value=2000.0,
    min=500.0,
    max=3000.0,
    step=50.0,
    description='Max Speed (mm/s):',
    layout=Layout(width='100%')
)

# Hatch spacing
hatch_spacing_mean = FloatSlider(
    value=0.1,
    min=0.05,
    max=0.5,
    step=0.01,
    description='Mean Hatch Spacing (mm):',
    layout=Layout(width='100%')
)

hatch_spacing_std = FloatSlider(
    value=0.01,
    min=0.0,
    max=0.05,
    step=0.001,
    description='Std Hatch Spacing (mm):',
    layout=Layout(width='100%')
)

# Region multipliers
contour_power_mult = FloatSlider(
    value=1.2,
    min=0.5,
    max=2.0,
    step=0.1,
    description='Contour Power Mult:',
    layout=Layout(width='100%')
)

contour_speed_mult = FloatSlider(
    value=0.8,
    min=0.5,
    max=2.0,
    step=0.1,
    description='Contour Speed Mult:',
    layout=Layout(width='100%')
)

support_power_mult = FloatSlider(
    value=0.7,
    min=0.5,
    max=2.0,
    step=0.1,
    description='Support Power Mult:',
    layout=Layout(width='100%')
)

support_speed_mult = FloatSlider(
    value=1.5,
    min=0.5,
    max=2.0,
    step=0.1,
    description='Support Speed Mult:',
    layout=Layout(width='100%')
)

laser_random_seed = IntText(
    value=None,
    description='Random Seed:',
    layout=Layout(width='200px')
)

preview_laser_params_button = Button(
    description='Preview Distribution',
    button_style='info',
    icon='chart-line',
    layout=Layout(width='180px')
)

laser_config_panel = VBox([
    WidgetHTML("<b>Laser Parameters Configuration</b>"),
    WidgetHTML("<i>Power Configuration</i>"),
    power_mean, power_std, power_min, power_max,
    WidgetHTML("<i>Speed Configuration</i>"),
    speed_mean, speed_std, speed_min, speed_max,
    WidgetHTML("<i>Hatch Spacing</i>"),
    hatch_spacing_mean, hatch_spacing_std,
    WidgetHTML("<i>Region Multipliers</i>"),
    contour_power_mult, contour_speed_mult,
    support_power_mult, support_speed_mult,
    HBox([laser_random_seed, preview_laser_params_button])
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))


In [12]:
# Part 5: ISPM Data Configuration

# Temperature configuration
base_temperature = FloatSlider(
    value=1500.0,
    min=1000.0,
    max=2000.0,
    step=10.0,
    description='Base Temperature (°C):',
    layout=Layout(width='100%')
)

temperature_variation = FloatSlider(
    value=50.0,
    min=0.0,
    max=200.0,
    step=5.0,
    description='Temp Variation (°C):',
    layout=Layout(width='100%')
)

# Melt pool size
melt_pool_width_mean = FloatSlider(
    value=0.5,
    min=0.1,
    max=2.0,
    step=0.05,
    description='Width Mean (mm):',
    layout=Layout(width='100%')
)

melt_pool_width_std = FloatSlider(
    value=0.1,
    min=0.0,
    max=0.5,
    step=0.01,
    description='Width Std (mm):',
    layout=Layout(width='100%')
)

melt_pool_length_mean = FloatSlider(
    value=0.8,
    min=0.1,
    max=3.0,
    step=0.05,
    description='Length Mean (mm):',
    layout=Layout(width='100%')
)

melt_pool_length_std = FloatSlider(
    value=0.15,
    min=0.0,
    max=0.5,
    step=0.01,
    description='Length Std (mm):',
    layout=Layout(width='100%')
)

melt_pool_depth_mean = FloatSlider(
    value=0.3,
    min=0.1,
    max=1.0,
    step=0.05,
    description='Depth Mean (mm):',
    layout=Layout(width='100%')
)

melt_pool_depth_std = FloatSlider(
    value=0.05,
    min=0.0,
    max=0.2,
    step=0.01,
    description='Depth Std (mm):',
    layout=Layout(width='100%')
)

# Thermal parameters
cooling_rate_mean = FloatSlider(
    value=100.0,
    min=10.0,
    max=500.0,
    step=5.0,
    description='Cooling Rate Mean (K/s):',
    layout=Layout(width='100%')
)

cooling_rate_std = FloatSlider(
    value=20.0,
    min=0.0,
    max=100.0,
    step=1.0,
    description='Cooling Rate Std (K/s):',
    layout=Layout(width='100%')
)

temp_gradient_mean = FloatSlider(
    value=50.0,
    min=10.0,
    max=200.0,
    step=5.0,
    description='Temp Gradient Mean (K/mm):',
    layout=Layout(width='100%')
)

temp_gradient_std = FloatSlider(
    value=10.0,
    min=0.0,
    max=50.0,
    step=1.0,
    description='Temp Gradient Std (K/mm):',
    layout=Layout(width='100%')
)

# Sampling
sampling_rate = FloatSlider(
    value=1000.0,
    min=100.0,
    max=10000.0,
    step=100.0,
    description='Sampling Rate (Hz):',
    layout=Layout(width='100%')
)

points_per_layer = IntSlider(
    value=1000,
    min=100,
    max=10000,
    step=100,
    description='Points Per Layer:',
    layout=Layout(width='100%')
)

ispm_random_seed = IntText(
    value=None,
    description='Random Seed:',
    layout=Layout(width='200px')
)

preview_thermal_button = Button(
    description='Preview Thermal Profile',
    button_style='info',
    icon='chart-line',
    layout=Layout(width='180px')
)

ispm_config_panel = VBox([
    WidgetHTML("<b>ISPM Data Configuration</b>"),
    WidgetHTML("<i>Temperature Configuration</i>"),
    base_temperature, temperature_variation,
    WidgetHTML("<i>Melt Pool Size</i>"),
    melt_pool_width_mean, melt_pool_width_std,
    melt_pool_length_mean, melt_pool_length_std,
    melt_pool_depth_mean, melt_pool_depth_std,
    WidgetHTML("<i>Thermal Parameters</i>"),
    cooling_rate_mean, cooling_rate_std,
    temp_gradient_mean, temp_gradient_std,
    WidgetHTML("<i>Sampling</i>"),
    sampling_rate, points_per_layer,
    HBox([ispm_random_seed, preview_thermal_button])
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))


In [13]:
# Part 6: CT Scan Configuration

# Grid dimensions
ct_x_dim = IntSlider(
    value=200,
    min=50,
    max=1000,
    step=10,
    description='X Dimension:',
    layout=Layout(width='100%')
)

ct_y_dim = IntSlider(
    value=200,
    min=50,
    max=1000,
    step=10,
    description='Y Dimension:',
    layout=Layout(width='100%')
)

ct_z_dim = IntSlider(
    value=200,
    min=50,
    max=1000,
    step=10,
    description='Z Dimension:',
    layout=Layout(width='100%')
)

# Voxel spacing
ct_x_spacing = FloatSlider(
    value=0.1,
    min=0.01,
    max=1.0,
    step=0.01,
    description='X Spacing (mm):',
    layout=Layout(width='100%')
)

ct_y_spacing = FloatSlider(
    value=0.1,
    min=0.01,
    max=1.0,
    step=0.01,
    description='Y Spacing (mm):',
    layout=Layout(width='100%')
)

ct_z_spacing = FloatSlider(
    value=0.1,
    min=0.01,
    max=1.0,
    step=0.01,
    description='Z Spacing (mm):',
    layout=Layout(width='100%')
)

# Material properties
base_density = FloatSlider(
    value=8.0,
    min=1.0,
    max=20.0,
    step=0.1,
    description='Base Density (g/cm³):',
    layout=Layout(width='100%')
)

density_variation = FloatSlider(
    value=0.2,
    min=0.0,
    max=2.0,
    step=0.1,
    description='Density Variation (g/cm³):',
    layout=Layout(width='100%')
)

base_porosity = FloatSlider(
    value=0.01,
    min=0.0,
    max=0.1,
    step=0.001,
    description='Base Porosity:',
    layout=Layout(width='100%')
)

porosity_variation = FloatSlider(
    value=0.005,
    min=0.0,
    max=0.05,
    step=0.001,
    description='Porosity Variation:',
    layout=Layout(width='100%')
)

# Defect parameters
defect_probability = FloatSlider(
    value=0.01,
    min=0.0,
    max=0.1,
    step=0.001,
    description='Defect Probability:',
    layout=Layout(width='100%')
)

defect_size_min = IntSlider(
    value=2,
    min=1,
    max=50,
    step=1,
    description='Defect Size Min (voxels):',
    layout=Layout(width='100%')
)

defect_size_max = IntSlider(
    value=10,
    min=2,
    max=100,
    step=1,
    description='Defect Size Max (voxels):',
    layout=Layout(width='100%')
)

defect_density_reduction = FloatSlider(
    value=0.3,
    min=0.0,
    max=0.8,
    step=0.05,
    description='Defect Density Reduction:',
    layout=Layout(width='100%')
)

# Scan quality
noise_level = FloatSlider(
    value=0.05,
    min=0.0,
    max=0.2,
    step=0.01,
    description='Noise Level:',
    layout=Layout(width='100%')
)

ct_random_seed = IntText(
    value=None,
    description='Random Seed:',
    layout=Layout(width='200px')
)

preview_ct_button = Button(
    description='Preview CT Scan',
    button_style='info',
    icon='cube',
    layout=Layout(width='180px')
)

ct_config_panel = VBox([
    WidgetHTML("<b>CT Scan Configuration</b>"),
    WidgetHTML("<i>Grid Dimensions</i>"),
    ct_x_dim, ct_y_dim, ct_z_dim,
    WidgetHTML("<i>Voxel Spacing</i>"),
    ct_x_spacing, ct_y_spacing, ct_z_spacing,
    WidgetHTML("<i>Material Properties</i>"),
    base_density, density_variation,
    base_porosity, porosity_variation,
    WidgetHTML("<i>Defect Parameters</i>"),
    defect_probability, defect_size_min, defect_size_max,
    defect_density_reduction,
    WidgetHTML("<i>Scan Quality</i>"),
    noise_level,
    HBox([ct_random_seed, preview_ct_button])
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))


In [14]:
# Part 7: MongoDB Population Options

# Collections to populate
collections_to_populate = SelectMultiple(
    options=[
        ('STL Models', 'stl_models'),
        ('Hatching Layers', 'hatching_layers'),
        ('Laser Parameters', 'laser_parameters'),
        ('ISPM Data', 'ispm_data'),
        ('CT Scan Data', 'ct_scan_data')
    ],
    value=['stl_models', 'hatching_layers', 'laser_parameters', 'ispm_data', 'ct_scan_data'],
    description='Collections:',
    layout=Layout(width='100%')
)

# Delete options
delete_all_toggle = Checkbox(
    value=False,
    description='Delete All Before Populate',
    layout=Layout(width='200px')
)

delete_by_model_id = Text(
    value='',
    placeholder='Enter model_id',
    description='Delete by Model ID:',
    layout=Layout(width='300px')
)

delete_by_model_button = Button(
    description='Delete',
    button_style='danger',
    icon='trash',
    layout=Layout(width='100px')
)

delete_by_collection = Dropdown(
    options=['stl_models', 'hatching_layers', 'laser_parameters', 'ispm_data', 'ct_scan_data'],
    value='stl_models',
    description='Delete Collection:',
    layout=Layout(width='200px')
)

delete_collection_button = Button(
    description='Delete',
    button_style='danger',
    icon='trash',
    layout=Layout(width='100px')
)

# Batch options
batch_size = IntSlider(
    value=100,
    min=1,
    max=1000,
    step=10,
    description='Batch Size:',
    layout=Layout(width='100%')
)

parallel_workers = IntSlider(
    value=4,
    min=1,
    max=20,
    step=1,
    description='Parallel Workers:',
    layout=Layout(width='100%')
)

# Storage options
compress_data = Checkbox(
    value=True,
    description='Compress Data',
    layout=Layout(width='150px')
)

index_collections = Checkbox(
    value=True,
    description='Index Collections',
    layout=Layout(width='150px')
)

store_metadata = Checkbox(
    value=True,
    description='Store Metadata',
    layout=Layout(width='150px')
)

mongodb_config_panel = VBox([
    WidgetHTML("<b>MongoDB Population Options</b>"),
    collections_to_populate,
    WidgetHTML("<i>Delete Options</i>"),
    delete_all_toggle,
    HBox([delete_by_model_id, delete_by_model_button]),
    HBox([delete_by_collection, delete_collection_button]),
    WidgetHTML("<i>Batch Options</i>"),
    batch_size, parallel_workers,
    WidgetHTML("<i>Storage Options</i>"),
    HBox([compress_data, index_collections, store_metadata])
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))


In [15]:
# Part 8: Combine Configuration Panels into Accordion

config_accordion = Accordion(children=[
    stl_config_panel,
    hatching_config_panel,
    laser_config_panel,
    ispm_config_panel,
    ct_config_panel,
    mongodb_config_panel
])

config_accordion.set_title(0, '1. STL Processing')
config_accordion.set_title(1, '2. Hatching Generation')
config_accordion.set_title(2, '3. Laser Parameters')
config_accordion.set_title(3, '4. ISPM Data')
config_accordion.set_title(4, '5. CT Scan')
config_accordion.set_title(5, '6. MongoDB Options')

# Display configuration accordion
display(config_accordion)


Accordion(children=(VBox(children=(HTML(value='<b>STL Processing Configuration</b>'), HBox(children=(Button(bu…

### Visualization Panel

Preview generated data using the tabbed visualization interface below. Each tab shows different aspects of the generated data.


In [16]:
# Part 9: Center Panel - Visualization Tabs

# Create output widgets for each visualization tab
stl_hatching_output = Output(layout=Layout(height='500px', border='1px solid #ccc'))
laser_params_output = Output(layout=Layout(height='500px', border='1px solid #ccc'))
ispm_output = Output(layout=Layout(height='500px', border='1px solid #ccc'))
ct_scan_output = Output(layout=Layout(height='500px', border='1px solid #ccc'))
preview_output = Output(layout=Layout(height='500px', border='1px solid #ccc'))

# Create tab widget
visualization_tabs = Tab(children=[
    stl_hatching_output,
    laser_params_output,
    ispm_output,
    ct_scan_output,
    preview_output
])

visualization_tabs.set_title(0, 'STL & Hatching')
visualization_tabs.set_title(1, 'Laser Parameters')
visualization_tabs.set_title(2, 'ISPM Data')
visualization_tabs.set_title(3, 'CT Scan')
visualization_tabs.set_title(4, 'Generation Preview')

# Initialize tabs with placeholder messages
with stl_hatching_output:
    display(HTML("<p><i>Load an STL file and generate hatching to see visualization</i></p>"))

with laser_params_output:
    display(HTML("<p><i>Generate laser parameters to see distribution plots</i></p>"))

with ispm_output:
    display(HTML("<p><i>Generate ISPM data to see thermal profiles</i></p>"))

with ct_scan_output:
    display(HTML("<p><i>Generate CT scan data to see volume visualization</i></p>"))

with preview_output:
    display(HTML("<p><i>Click 'Preview' to see combined data preview</i></p>"))

display(visualization_tabs)


### Status and Results Panel

Monitor generation progress, view statistics, and check MongoDB status.


In [ ]:
# Part 10: Right Panel - Status and Results

# Generation status
current_operation = WidgetHTML(value='<b>Status:</b> Ready')
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)

# Step progress indicators
step_status_html = WidgetHTML(value='''
<table style="width:100%">
<tr><td>STL Processing:</td><td>⏳</td></tr>
<tr><td>Hatching:</td><td>⏳</td></tr>
<tr><td>Laser Parameters:</td><td>⏳</td></tr>
<tr><td>ISPM Data:</td><td>⏳</td></tr>
<tr><td>CT Scan:</td><td>⏳</td></tr>
<tr><td>MongoDB Population:</td><td>⏳</td></tr>
</table>
''')

# Generation statistics
gen_stats_html = WidgetHTML(value='''
<b>Generation Statistics</b><br>
<i>No data generated yet</i>
''')

# MongoDB status
mongo_status_html = WidgetHTML(value='''
<b>MongoDB Status</b><br>
<i>Checking connection...</i>
''')

refresh_mongo_button = Button(
    description='Refresh Status',
    button_style='info',
    icon='refresh',
    layout=Layout(width='150px')
)

# Export options
export_config_button = Button(
    description='Export Config',
    button_style='',
    icon='download',
    layout=Layout(width='150px')
)

export_preview_button = Button(
    description='Export Preview',
    button_style='',
    icon='download',
    layout=Layout(width='150px')
)

export_stats_button = Button(
    description='Export Statistics',
    button_style='',
    icon='download',
    layout=Layout(width='150px')
)

# Right panel layout
status_panel = VBox([
    WidgetHTML("<b>Generation Status</b>"),
    current_operation,
    progress_bar,
    step_status_html,
    HTML("<hr>"),
    gen_stats_html,
    HTML("<hr>"),
    WidgetHTML("<b>MongoDB Status</b>"),
    mongo_status_html,
    refresh_mongo_button,
    HTML("<hr>"),
    WidgetHTML("<b>Export Options</b>"),
    export_config_button,
    export_preview_button,
    export_stats_button
], layout=Layout(
    padding='10px',
    border='1px solid #ccc',
    margin='5px',
    width='300px'
))

display(status_panel)


In [ ]:
# Part 11: Bottom Panel - Status and Logs

bottom_status = WidgetHTML(value='<b>Status:</b> Ready | <b>Progress:</b> 0% | <b>Time:</b> 0:00')
bottom_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Overall:',
    bar_style='info',
    layout=Layout(width='100%')
)

generation_logs = Output(layout=Layout(
    height='200px',
    border='1px solid #ccc',
    overflow_y='auto'
))

# Initialize logs
with generation_logs:
    display(HTML("<p><i>Generation logs will appear here...</i></p>"))

bottom_panel = VBox([
    bottom_status,
    bottom_progress,
    WidgetHTML("<b>Generation Logs:</b>"),
    generation_logs
], layout=Layout(
    padding='10px',
    border='1px solid #ddd',
    margin='5px'
))

display(bottom_panel)


In [ ]:
# Part 12: Main Layout Assembly

# Create main layout with left (config), center (visualization), right (status) panels
main_layout = HBox([
    # Left panel: Configuration accordion (30% width)
    VBox([config_accordion], layout=Layout(width='30%', padding='5px')),
    
    # Center panel: Visualization tabs (50% width)
    VBox([visualization_tabs], layout=Layout(width='50%', padding='5px')),
    
    # Right panel: Status and results (20% width)
    VBox([status_panel], layout=Layout(width='20%', padding='5px'))
], layout=Layout(
    width='100%',
    height='600px',
    border='2px solid #333',
    padding='10px'
))

# Complete interface
complete_interface = VBox([
    top_panel,
    main_layout,
    bottom_panel
], layout=Layout(width='100%'))

# Complete interface assembled (will be displayed in next cell)


In [ ]:
# Display the complete interface
display(complete_interface)
print("✅ Interface displayed - ready to use!")


## Event Handlers and Generation Logic

The following cells implement the event handlers and generation functions that connect the widgets to the actual data generation modules.


In [ ]:
# Part 13: Global State and Helper Functions

# Global state for generated data
generated_data = {
    'stl_model': None,
    'hatching_data': None,
    'laser_params': None,
    'ispm_data': None,
    'ct_scan': None,
    'model_id': None
}

# Generation start time
generation_start_time = None

def log_message(message: str, level: str = 'info'):
    """Log a message to the generation logs."""
    timestamp = datetime.now().strftime('%H:%M:%S')
    icon = {'info': 'ℹ️', 'success': '✅', 'warning': '⚠️', 'error': '❌'}.get(level, 'ℹ️')
    
    with generation_logs:
        print(f"[{timestamp}] {icon} {message}")

def update_status(operation: str, progress: int = None):
    """Update the status display."""
    current_operation.value = f'<b>Status:</b> {operation}'
    if progress is not None:
        progress_bar.value = progress
        bottom_progress.value = progress

def update_step_status(step: str, status: str):
    """Update step status indicator."""
    icons = {'pending': '⏳', 'running': '🔄', 'success': '✅', 'error': '❌'}
    # This would update the step_status_html table
    # Implementation would parse and update the HTML table

def check_generation_ready() -> bool:
    """Check if generation is ready (STL loaded and basic config set)."""
    # Check if STL is loaded
    if generated_data.get('stl_model') is None:
        return False
    
    # Check if STL file is selected
    if not stl_selector.value or stl_selector.value == 'No STL files found':
        return False
    
    # Basic validation - at least one collection should be selected
    if len(collections_to_populate.value) == 0:
        return False
    
    return True

def update_generate_button_state():
    """Update Generate button enabled/disabled state based on readiness."""
    is_ready = check_generation_ready()
    generate_button.disabled = not is_ready
    
    if is_ready:
        generate_button.tooltip = 'Ready to generate data'
        generate_button.button_style = 'success'
        generate_status_label.value = '<i style="color:green; font-size:0.9em;">✅ Ready to generate</i>'
    else:
        generate_button.tooltip = 'Load STL file and configure parameters first'
        generate_button.button_style = ''
        if generated_data.get('stl_model') is None:
            generate_status_label.value = '<i style="color:gray; font-size:0.9em;">⚠️ Load STL file to enable Generate</i>'
        elif len(collections_to_populate.value) == 0:
            generate_status_label.value = '<i style="color:gray; font-size:0.9em;">⚠️ Select at least one collection to populate</i>'
        else:
            generate_status_label.value = '<i style="color:gray; font-size:0.9em;">⚠️ Configure parameters to enable Generate</i>'

def get_current_config() -> Dict[str, Any]:
    """Collect all current widget values into a configuration dictionary."""
    return {
        'generation_mode': generation_mode.value,
        'stl_file': stl_selector.value,
        'hatching': {
            'layer_thickness': layer_thickness.value,
            'hatch_spacing': hatch_spacing.value,
            'hatch_angle': hatch_angle.value,
            'contour_offset': contour_offset.value,
            'max_layers': max_layers.value,
            'sampling_spacing': sampling_spacing.value,
            'laser_beam_width': laser_beam_width.value,
            'pattern': hatch_pattern.value
        },
        'laser': {
            'power_mean': power_mean.value,
            'power_std': power_std.value,
            'power_min': power_min.value,
            'power_max': power_max.value,
            'speed_mean': speed_mean.value,
            'speed_std': speed_std.value,
            'speed_min': speed_min.value,
            'speed_max': speed_max.value,
            'hatch_spacing_mean': hatch_spacing_mean.value,
            'hatch_spacing_std': hatch_spacing_std.value,
            'contour_power_mult': contour_power_mult.value,
            'contour_speed_mult': contour_speed_mult.value,
            'support_power_mult': support_power_mult.value,
            'support_speed_mult': support_speed_mult.value,
            'random_seed': laser_random_seed.value if laser_random_seed.value else None
        },
        'ispm': {
            'base_temperature': base_temperature.value,
            'temperature_variation': temperature_variation.value,
            'melt_pool_width_mean': melt_pool_width_mean.value,
            'melt_pool_width_std': melt_pool_width_std.value,
            'melt_pool_length_mean': melt_pool_length_mean.value,
            'melt_pool_length_std': melt_pool_length_std.value,
            'melt_pool_depth_mean': melt_pool_depth_mean.value,
            'melt_pool_depth_std': melt_pool_depth_std.value,
            'cooling_rate_mean': cooling_rate_mean.value,
            'cooling_rate_std': cooling_rate_std.value,
            'temp_gradient_mean': temp_gradient_mean.value,
            'temp_gradient_std': temp_gradient_std.value,
            'sampling_rate': sampling_rate.value,
            'points_per_layer': points_per_layer.value,
            'random_seed': ispm_random_seed.value if ispm_random_seed.value else None
        },
        'ct_scan': {
            'grid_dimensions': (ct_x_dim.value, ct_y_dim.value, ct_z_dim.value),
            'voxel_spacing': (ct_x_spacing.value, ct_y_spacing.value, ct_z_spacing.value),
            'base_density': base_density.value,
            'density_variation': density_variation.value,
            'base_porosity': base_porosity.value,
            'porosity_variation': porosity_variation.value,
            'defect_probability': defect_probability.value,
            'defect_size_range': (defect_size_min.value, defect_size_max.value),
            'defect_density_reduction': defect_density_reduction.value,
            'noise_level': noise_level.value,
            'random_seed': ct_random_seed.value if ct_random_seed.value else None
        },
        'mongodb': {
            'collections': list(collections_to_populate.value),
            'delete_all': delete_all_toggle.value,
            'batch_size': batch_size.value,
            'parallel_workers': parallel_workers.value,
            'compress': compress_data.value,
            'index': index_collections.value,
            'store_metadata': store_metadata.value
        }
    }

print("✅ Helper functions defined")


In [ ]:
# Part 14: STL Loading Handler

def on_stl_load_clicked(b):
    """Handle STL load button click."""
    if not GENERATION_AVAILABLE:
        log_message("Generation modules not available", 'error')
        return
    
    stl_filename = stl_selector.value
    if not stl_filename or stl_filename == 'No STL files found':
        log_message("No STL file selected", 'warning')
        return
    
    try:
        log_message(f"Loading STL file: {stl_filename}", 'info')
        update_status("Loading STL...", 10)
        
        # Initialize STL processor
        stl_processor = STLProcessor()
        stl_file_path = stl_processor.get_stl_file(stl_filename)
        
        if stl_file_path and stl_file_path.exists():
            # Load STL using pyslm if available
            try:
                import pyslm
                stl_part = pyslm.Part(stl_file_path)
                
                # Extract metadata
                bbox = stl_part.boundingBox
                metadata = {
                    'filename': stl_filename,
                    'path': str(stl_file_path),
                    'bounding_box': {
                        'x_min': bbox[0], 'x_max': bbox[3],
                        'y_min': bbox[1], 'y_max': bbox[4],
                        'z_min': bbox[2], 'z_max': bbox[5]
                    },
                    'volume': stl_part.volume if hasattr(stl_part, 'volume') else None,
                    'loaded_at': datetime.now().isoformat()
                }
                
                generated_data['stl_model'] = stl_part
                generated_data['model_id'] = stl_filename.replace('.stl', '')
                
                # Update metadata display
                metadata_text = json.dumps(metadata, indent=2)
                stl_metadata_display.value = metadata_text
                
                log_message(f"STL loaded successfully: {stl_filename}", 'success')
                update_status("STL loaded", 20)
                
                # Update Generate button state
                update_generate_button_state()
                
                # Show 3D preview if enabled
                if stl_preview_toggle.value:
                    with stl_hatching_output:
                        clear_output(wait=True)
                        display(HTML(f"<p><b>STL Model:</b> {stl_filename}</p>"))
                        display(HTML(f"<p><b>Bounding Box:</b> X: {bbox[0]:.2f} to {bbox[3]:.2f}, "
                                    f"Y: {bbox[1]:.2f} to {bbox[4]:.2f}, "
                                    f"Z: {bbox[2]:.2f} to {bbox[5]:.2f} mm</p>"))
                        if metadata['volume']:
                            display(HTML(f"<p><b>Volume:</b> {metadata['volume']:.2f} mm³</p>"))
                        display(HTML("<p><i>3D visualization would appear here (requires PyVista)</i></p>"))
                
            except ImportError:
                log_message("pyslm not available - limited STL processing", 'warning')
                metadata = {'filename': stl_filename, 'path': str(stl_file_path)}
                stl_metadata_display.value = json.dumps(metadata, indent=2)
        else:
            log_message(f"STL file not found: {stl_filename}", 'error')
            update_status("Error: STL file not found", 0)
            
    except Exception as e:
        log_message(f"Error loading STL: {str(e)}", 'error')
        update_status(f"Error: {str(e)}", 0)

# Attach handler
stl_load_button.on_click(on_stl_load_clicked)
print("✅ STL loading handler attached")

# Add observer to collections selector to update button state
def on_collections_change(change):
    """Update Generate button when collections selection changes."""
    update_generate_button_state()

collections_to_populate.observe(on_collections_change, names='value')
print("✅ Collection selector observer attached")

# Initialize Generate button state
update_generate_button_state()
print("✅ Generate button state initialized")


In [ ]:
# Part 15: Preview Button Handler

def on_preview_clicked(b):
    """Handle preview button click - generate and preview data without saving."""
    if not GENERATION_AVAILABLE:
        log_message("Generation modules not available", 'error')
        return
    
    log_message("Starting preview generation...", 'info')
    update_status("Generating preview...", 5)
    
    try:
        config = get_current_config()
        
        # Generate preview data (simplified - would call actual generators)
        with preview_output:
            clear_output(wait=True)
            display(HTML("<h3>Generation Preview</h3>"))
            display(HTML(f"<p><b>Mode:</b> {config['generation_mode']}</p>"))
            display(HTML(f"<p><b>STL File:</b> {config['stl_file']}</p>"))
            display(HTML("<p><b>Configuration Summary:</b></p>"))
            display(HTML(f"<ul>"))
            display(HTML(f"<li>Hatching: {config['hatching']['max_layers']} layers, "
                       f"{config['hatching']['layer_thickness']} mm thickness</li>"))
            display(HTML(f"<li>Laser: {config['laser']['power_mean']} W mean power, "
                       f"{config['laser']['speed_mean']} mm/s mean speed</li>"))
            display(HTML(f"<li>ISPM: {config['ispm']['points_per_layer']} points/layer, "
                       f"{config['ispm']['sampling_rate']} Hz sampling</li>"))
            display(HTML(f"<li>CT Scan: {config['ct_scan']['grid_dimensions']} grid, "
                       f"{config['ct_scan']['voxel_spacing'][0]} mm spacing</li>"))
            display(HTML(f"</ul>"))
            display(HTML("<p><i>Full preview would show generated data visualizations</i></p>"))
        
        log_message("Preview generated successfully", 'success')
        update_status("Preview ready", 100)
        
    except Exception as e:
        log_message(f"Error generating preview: {str(e)}", 'error')
        update_status(f"Error: {str(e)}", 0)

# Attach handler
preview_button.on_click(on_preview_clicked)
print("✅ Preview handler attached")


In [ ]:
# Part 16: Generate Button Handler (Placeholder - Full implementation would call generation modules)

def on_generate_clicked(b):
    """Handle generate button click - full data generation and MongoDB population."""
    if not GENERATION_AVAILABLE:
        log_message("Generation modules not available", 'error')
        return
    
    if not mongo_client or not mongo_client.is_connected():
        log_message("MongoDB not connected", 'error')
        update_status("Error: MongoDB not connected", 0)
        return
    
    global generation_start_time
    generation_start_time = time.time()
    
    log_message("Starting full data generation...", 'info')
    update_status("Generating data...", 10)
    
    try:
        config = get_current_config()
        
        # This is a placeholder - full implementation would:
        # 1. Load and process STL
        # 2. Generate hatching paths
        # 3. Generate laser parameters
        # 4. Generate ISPM data
        # 5. Generate CT scan data
        # 6. Populate MongoDB collections
        
        log_message("Generation complete (placeholder)", 'success')
        update_status("Generation complete", 100)
        
        # Update statistics
        gen_stats_html.value = '''
        <b>Generation Statistics</b><br>
        <ul>
        <li>STL Models: 1</li>
        <li>Hatching Layers: Generated</li>
        <li>Laser Parameters: Generated</li>
        <li>ISPM Data Points: Generated</li>
        <li>CT Scan Voxels: Generated</li>
        </ul>
        '''
        
    except Exception as e:
        log_message(f"Error during generation: {str(e)}", 'error')
        update_status(f"Error: {str(e)}", 0)

# Attach handler
generate_button.on_click(on_generate_clicked)
print("✅ Generate handler attached")


In [ ]:
# Part 17: MongoDB Status Refresh Handler

def on_refresh_mongo_clicked(b):
    """Refresh MongoDB status display."""
    if not mongo_client or not mongo_client.is_connected():
        mongo_status_html.value = '''
        <b>MongoDB Status</b><br>
        <span style="color:red">❌ Not Connected</span>
        '''
        return
    
    try:
        db = mongo_client.database
        collections = db.list_collection_names()
        
        # Get collection stats
        stats = []
        total_docs = 0
        for coll_name in collections:
            coll = db[coll_name]
            count = coll.count_documents({})
            total_docs += count
            stats.append(f"<tr><td>{coll_name}</td><td>{count:,}</td></tr>")
        
        mongo_status_html.value = f'''
        <b>MongoDB Status</b><br>
        <span style="color:green">✅ Connected</span><br>
        <b>Database:</b> {db.name}<br>
        <b>Collections:</b> {len(collections)}<br>
        <b>Total Documents:</b> {total_docs:,}<br>
        <table style="width:100%; font-size:0.9em">
        <tr><th>Collection</th><th>Documents</th></tr>
        {''.join(stats)}
        </table>
        '''
        
        log_message("MongoDB status refreshed", 'success')
        
    except Exception as e:
        mongo_status_html.value = f'''
        <b>MongoDB Status</b><br>
        <span style="color:red">❌ Error: {str(e)}</span>
        '''
        log_message(f"Error refreshing MongoDB status: {str(e)}", 'error')

# Attach handler and do initial refresh
refresh_mongo_button.on_click(on_refresh_mongo_clicked)
on_refresh_mongo_clicked(None)  # Initial refresh
print("✅ MongoDB status handler attached")


In [ ]:
# Part 18: Save Configuration Handler

def on_save_config_clicked(b):
    """Save current configuration to JSON file."""
    try:
        config = get_current_config()
        config['saved_at'] = datetime.now().isoformat()
        
        # Create filename with timestamp
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"generation_config_{timestamp}.json"
        
        # Save to file
        config_file = project_root / filename
        with open(config_file, 'w') as f:
            json.dump(config, f, indent=2)
        
        log_message(f"Configuration saved to {filename}", 'success')
        
        # Show download link
        with generation_logs:
            display(HTML(f'<p>✅ Configuration saved: <code>{filename}</code></p>'))
            
    except Exception as e:
        log_message(f"Error saving configuration: {str(e)}", 'error')

# Attach handler
save_config_button.on_click(on_save_config_clicked)
print("✅ Save config handler attached")


In [ ]:
# Part 19: Clear Preview Handler

def on_clear_preview_clicked(b):
    """Clear all preview visualizations."""
    with stl_hatching_output:
        clear_output(wait=True)
        display(HTML("<p><i>Load an STL file and generate hatching to see visualization</i></p>"))
    
    with laser_params_output:
        clear_output(wait=True)
        display(HTML("<p><i>Generate laser parameters to see distribution plots</i></p>"))
    
    with ispm_output:
        clear_output(wait=True)
        display(HTML("<p><i>Generate ISPM data to see thermal profiles</i></p>"))
    
    with ct_scan_output:
        clear_output(wait=True)
        display(HTML("<p><i>Generate CT scan data to see volume visualization</i></p>"))
    
    with preview_output:
        clear_output(wait=True)
        display(HTML("<p><i>Click 'Preview' to see combined data preview</i></p>"))
    
    log_message("Preview cleared", 'info')

# Attach handler
clear_preview_button.on_click(on_clear_preview_clicked)
print("✅ Clear preview handler attached")


y
